In [1]:
import os
import numpy as np
import pandas as pd
from custom import utils
from collections import Counter
import inspect
import scipy.io

In [2]:
def lookup(x):
    print(inspect.getsource(x))
    return None
!ls

170_columns.txt			  covars_ecn.txt
2k_columns.txt			  covars_no_ecn.txt
bayes_reg.m			  cv_idxs.mat
bf_1				  data_prep.ipynb
bf_2				  FXVB_py.ipynb
bf_2_respaths			  genomic_N1547_P100006_matched.hdf5py
bf_3				  id_variables.txt
brain_gene.mat			  KCCA.ipynb
brain_N1547_P5927_matched.hdf5py  pyscript.m


# data that will be used for bayes script, still need to subset it

In [3]:
brain_data = pd.read_hdf("brain_N1547_P5927_matched.hdf5py")
genomic_data = pd.read_hdf("genomic_N1547_P100006_matched.hdf5py")
print("Brain data shape: {}".format(brain_data.shape))
print("Genomic data shape: {}".format(genomic_data.shape))

Brain data shape: (1547, 5927)
Genomic data shape: (1547, 100006)


# text files for subsetting some of the data, column wise

In [4]:
b170 = np.genfromtxt("170_columns.txt", dtype=str)
cvars_descrete = np.genfromtxt("covars_ecn.txt", dtype=str)
cvars_continuous = np.genfromtxt("covars_no_ecn.txt")

# paranoia check for matching IDs

In [5]:
assert((brain_data.IID.values == genomic_data.IID.values).mean() == 1.)

# further preprocessing

In [6]:
group = brain_data.GROUP.values
cnt_scz = np.logical_or(group == 'Control', group == 'Schizophrenia')
brain = brain_data.ix[cnt_scz, :]
genomic = genomic_data.ix[cnt_scz, :]
y_g = genomic.DIAG.values - 1
y_b = np.array([0 if i == 'Control' else 1 for i in brain.GROUP.values])

# remove covariates from brain data

In [7]:
sex = genomic['SEX'].values
age = brain['AGE_MRI'].values
# there is one missing value for age, replacing that with the column mean
age_imputed = np.nan_to_num(age)
age[age_imputed == 0] = age[age_imputed != 0].mean()
ICV = brain['EstimatedTotalIntraCranialVol'].values
cvars_cont = np.hstack((sex[:, None], age[:, None], ICV[:, None]))
cvars_desc = np.hstack([utils.encoder(brain[col]) for col in cvars_descrete[1:]])
cvars_desc = utils.make_non_singular(utils.encoder(brain.STUDY.values))
cvars_joined = utils.make_non_singular(np.hstack((cvars_cont, cvars_desc)))
brain_170 = brain[b170]
X = utils.proj(brain_170.values, cvars_joined)
y = y_b.copy()
G = utils.proj(genomic.ix[:, 1:-5].values, cvars_joined)
snp_ids = genomic.columns[1:-5].values

# this is the file that will be used in the bayes factor step

In [8]:
scipy.io.savemat("brain_gene.mat",
                 mdict={'G': G, 
                        'I': X, 
                        'Z': np.array([]),
                        'colnames': snp_ids,
                        'y': y})

# starting the bayes factor workflow

In [18]:
import os
import numpy as np
from nipype import Function, Node, Workflow, IdentityInterface

In [19]:
wf = Workflow(name='brain_bcv')
wf.base_dir = "/om/user/ysa"

In [20]:
Iternode = Node(IdentityInterface(fields=['col_idx', 'cv_idx']), name = 'Iternode')
Iternode.iterables = [('col_idx', np.arange(170) + 1), ('cv_idx', np.arange(10) + 1)]

# creates the cross validation

In [21]:
def cv_maker(data_path, save_path):
    import scipy.io
    from sklearn.model_selection import StratifiedKFold
    X = scipy.io.loadmat(data_path)['I']
    y = scipy.io.loadmat(data_path)['y'][0]
    cv = StratifiedKFold(n_splits=10, random_state=1)
    train_idx, test_idx = {}, {}
    for idx, (train, test) in enumerate(cv.split(X, y)):
        train_idx['train_{}'.format(idx + 1)] = train + 1
        test_idx['test_{}'.format(idx + 1)] = test + 1
    scipy.io.savemat(save_path, mdict={"train":train_idx, "test":test_idx})
    return save_path
 
CV_maker = Node(interface=Function(
    input_names = ['data_path', 'save_path'],
    output_names = ['save_path'],
    function = cv_maker
), name = 'CV_maker')

CV_maker.inputs.data_path = "/storage/gablab001/data/genus/brain_genomic_bayes/brain_gene.mat"
CV_maker.inputs.save_path = "/storage/gablab001/data/genus/brain_genomic_bayes/cv_idxs.mat"

# bayes factor step (1)

In [22]:
def run_bayes(in_file, cv_file, cv_idx, col_idx, out_file):
    import cPickle as pickle
    import numpy as np
    import os
    import nipype.interfaces.matlab as Matlab
    def outnames(col, out):
        return os.path.join(out, '{}.mat'.format(col))
    col_names = np.genfromtxt("/storage/gablab001/data/genus/brain_genomic_bayes/170_columns.txt", dtype=str)
    col_save_name = col_names[col_idx - 1] + "_{}_{}_BF".format(cv_idx, col_idx)
    with open("/storage/gablab001/data/genus/brain_genomic_bayes/bayes_reg.m", "r") as src:
        script = src.read().replace("\n", "")
    mat_file = outnames(in_file[:-4] + col_save_name, out_file)
    matlab = Matlab.MatlabCommand()
    matlab.inputs.paths = [
    '/storage/gablab001/data/genus/current/variational_bayes_wrap/varbvs/varbvs-MATLAB',
    '/storage/gablab001/data/genus/current/variational_bayes_wrap/varbvs',
    '/storage/gablab001/data/genus/current/variational_bayes_wrap/varbvs/varbvs-R']
    matlab.inputs.script = script.format(in_file, cv_file, cv_idx, col_idx, mat_file)
    res = matlab.run()
    return mat_file

Run_bayes = Node(interface=Function(
    input_names = ['in_file', 'cv_file', 
                   'cv_idx', 'col_idx',
                   'out_file'],
    output_names = ['mat_file'],
    function = run_bayes
), name='Run_bayes')

Run_bayes.inputs.in_file = "/storage/gablab001/data/genus/brain_genomic_bayes/brain_gene.mat"
Run_bayes.inputs.out_file = "/storage/gablab001/data/genus/brain_genomic_bayes/"

# run step 1

In [1]:
wf.connect(CV_maker, 'save_path', Run_bayes, 'cv_file')
wf.connect(Iternode, 'cv_idx', Run_bayes, 'cv_idx')
wf.connect(Iternode, 'col_idx', Run_bayes, 'col_idx')
#wf.run(plugin='SLURM', plugin_args={'sbatch_args':'--mem=12G -t 3-23:00:00', 'max_jobs': 200})

NameError: name 'wf' is not defined

In [24]:
col_names = np.genfromtxt("/storage/gablab001/data/genus/brain_genomic_bayes/170_columns.txt", dtype=str)
inf = "/storage/gablab001/data/genus/brain_genomic_bayes/brain_gene.mat"
outf = "/storage/gablab001/data/genus/brain_genomic_bayes/bf_3/"

# addon functions for step 2 (fixed form VB)

In [25]:
def make_name(BF_input, col_idx, cv_idx, col_names):
    return (BF_input[:-4].split("/")[-1] + 
            col_names[col_idx - 1] + 
            "_{}_{}_BF.mat".format(cv_idx, col_idx))

def csv(colnum, outfile):
    import pandas as pd
    import os
    df = pd.DataFrame(columns=['colNum', 'matFn'])
    for i in range(1, colnum):
        df.loc[i] = [i, outfile+'{}.mat'.format(i)]
    df.iloc[:,0] = df.iloc[:,0].astype(int)
    return df

def write_csv(cv_idx, col_names, in_file, save_path):
    idx = np.arange(len(col_names))+1
    df = pd.DataFrame(columns=['colNum', 'matFn'], index=idx)
    df['colNum'] = idx
    paths = [os.path.join(save_path, make_name(in_file, i, cv_idx, col_names)) for i in idx]
    df['matFn'] = paths
    bf_2_respaths = "/storage/gablab001/data/genus/brain_genomic_bayes/bf_3_respaths"
    to_save = os.path.join(bf_2_respaths, "BFRESULTSLIST_{}.csv".format(cv_idx))
    df.to_csv(to_save, index=None)
    return to_save

# creates text files with paths to files written in step 1

In [26]:
for cv_idx in np.arange(10)+1:
    write_csv(
        cv_idx, 
        col_names, 
        inf, 
        "/storage/gablab001/data/genus/brain_genomic_bayes/bf_3"
    )

In [33]:
def run_fixedformVB():
    import cPickle as pickle
    import numpy as np
    import os
    import nipype.interfaces.matlab as Matlab
    matlab = Matlab.MatlabCommand()
    matlab.inputs.paths = [
        '/storage/gablab001/data/genus/bayes_adni/basis/bayesianImagingGenetics/src',
        '/storage/gablab001/data/genus/bayes_adni/basis/bayesianImagingGenetics/src/Utils']
    matlab.inputs.script = """
    deployEndoPhenVB('step','fxvb',...
                    'csvFile','/storage/gablab001/data/genus/brain_genomic_bayes/bf_3_respaths/BFRESULTSLIST_1.csv',...
                    'inputMat','/storage/gablab001/data/genus/brain_genomic_bayes/brain_gene.mat',...
                    'outFile','/om/user/ysa/ffvb_test.mat')
    """
    res = matlab.run()
    

In [34]:
run_fixedformVB()

RuntimeError: Command:
matlab -nodesktop -nosplash -singleCompThread -r "addpath('/storage/gablab001/data/genus/brain_genomic_bayes');pyscript;exit"
Standard output:
MATLAB is selecting SOFTWARE OPENGL rendering.

                            < M A T L A B (R) >
                  Copyright 1984-2016 The MathWorks, Inc.
                   R2016b (9.1.0.441655) 64-bit (glnxa64)
                             September 7, 2016

 
To get started, type one of these: helpwin, helpdesk, or demo.
For product information, visit www.mathworks.com.
 
Executing pyscript at 26-May-2017 19:00:00:
----------------------------------------------------------------------------------------------------
MATLAB Version: 9.1.0.441655 (R2016b)
MATLAB License Number: 650662
Operating System: Linux 3.10.0-229.el7.x86_64 #1 SMP Fri Mar 6 11:36:42 UTC 2015 x86_64
Java Version: Java 1.7.0_60-b19 with Oracle Corporation Java HotSpot(TM) 64-Bit Server VM mixed mode
-------------------------------------------------------

In [105]:
import scipy.io

In [106]:
scipy.io.loadmat("/storage/gablab001/data/genus/brain_genomic_bayes/brain_gene.mat").keys()

['Z',
 'G',
 'I',
 '__header__',
 '__globals__',
 'colnames',
 '__version__',
 'response']

In [40]:
b = scipy.io.loadmat("/storage/gablab001/data/genus/brain_genomic_bayes/bf_3/brain_genelh_Thalamus.Proper_1_168_BF.mat")

In [42]:
for key in b.keys():
    try:
        print(b[key].shape, key)
    except:
        pass

((100000, 9), 'mu')
((1, 9), 'lnZ')
((1, 1), 'lnZ_weighted')
((1, 9), 'w')
((100000, 1), 'PIP')
((100000, 9), 'alpha')


In [121]:
for k in ['mu', 'w', 'PIP', 'alpha']:
    print b[k].shape, k

(100000, 9) mu
(1, 9) w
(100000, 1) PIP
(100000, 9) alpha


In [32]:
b['lnZ']

array([[-8549.85556399, -8549.86356402, -8549.87174834, -8549.88107747,
        -8549.88980515, -8549.91771467, -8549.9586837 , -8550.03601001,
        -8550.17060025]])

0